In [1]:
# useing this variable for saving models and summaries
exp_name = "CNN_cpd_64"

## Importing Libraries

In [2]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
import numpy as np
from hdf5storage import savemat
import hickle 

import sys
sys.path.append('../src')

In [3]:
import models.CNN_model as CNN_model
import training
import evaluation.nn_eval as nn_eval

## Loading synthetic data

In [4]:
#loading data
data_path= '../data/opp7.hkl'
with open(data_path, 'r') as fin:
    data = hickle.load(fin)

In [5]:
train_data = data['train_data']
validation_data = data['validation_data']
test_data = data['test_data']

train_gt = data['train_gt']
validation_gt = data['validation_gt']
test_gt = data['test_gt']

In [6]:
train_data.shape, train_gt.shape

((1358, 1024, 77), (1358, 1024, 1))

In [7]:
test_data.shape, test_gt.shape

((233, 1024, 77), (233, 1024, 1))

## Model

In [8]:
# Model parameters
n_variables = 77
learning_rate = 0.001
batch_size = 100

convolution_levels = 3    #number of convolution to have in the network
pooling_stride = [4, 4, 4]
weights_shape = [[9, n_variables, 32],
                 [5, 32, 32],
                 [5, 32, 32]]
bias_shape = [32, 32, 32]
activation = tf.nn.relu
n_classes = 1

##positive weight for weighted cross entropy
s = train_gt.shape
resolution = np.prod(pooling_stride)

print s[0]*s[1], (s[0]*s[1]/resolution), (s[0]*s[1]/resolution)*resolution

pooled_gt = np.reshape(train_gt[:,:(s[0]*s[1]/resolution)*resolution,:], (s[0]*(s[1]/resolution), resolution, s[2])).max(axis=1)
pos_weight = (1.0 - pooled_gt.mean(axis = 0)) / pooled_gt.mean(axis=0)


1390592 21728 1390592


In [9]:
pos_weight.shape

(1,)

In [10]:
pos_weight

array([ 2.15172601], dtype=float32)

In [11]:
# Launch the graph
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [12]:
summary_path = '../summary/' + exp_name 
checkpoint_path = '../model/' + exp_name + '/' + exp_name 

In [13]:
model = CNN_model.Model(n_variables = n_variables,
                        learning_rate = learning_rate,
                        n_classes = n_classes)

In [14]:
model.build_conv_net(pos_weight_values = pos_weight,
                     learning_rate = learning_rate,
                     convolution_levels = convolution_levels, 
                     pooling_stride = pooling_stride,
                     weights_shape = weights_shape,
                     bias_shape = bias_shape,
                     n_classes = n_classes,
                     activation = activation,
                     trainable_conv = False,
                     trainable_last=True)

In [15]:
trainer = training.Trainer(model = model,
                    data_train = train_data,
                    ground_truth_train = train_gt,
                    data_validation = test_data,
                    ground_truth_validation = test_gt,
                    sess = sess,
                    summary_path = summary_path,
                    batch_size = batch_size,
                    global_step = 0,
                    eval_type = 'change')

In [16]:
#load the model
saver = tf.train.Saver(var_list = [
    model.weights['wc0'],model.weights['wc1'],model.weights['wc2'],
    model.bias['bc0'],model.bias['bc1'],model.bias['bc2']
])
saver.restore(sess, '../model/CNN9005_opp/CNN9005_opp-8344')

INFO:tensorflow:Restoring parameters from ../model/CNN9005_opp/CNN9005_opp-8344


In [17]:
trainer.train(max_iter = 20000,
                train_eval_step = 100, 
                validation_eval_step = 100,
                display_step = 100)

Train Batch Evaluation
AUC = 0.471 vs. 0.421
F1 = 0.576 vs. 0.581
Precision = 0.404 vs. 0.410
Recall = 1.000 vs. 0.994
Validation
AUC = 0.207 vs. 0.195
F1 = 0.326 vs. 0.317
Precision = 0.203 vs. 0.188
Recall = 0.828 vs. 1.000
Minibatch Loss= 49.830326
Train Batch Evaluation
AUC = 0.443 vs. 0.271
F1 = 0.472 vs. 0.437
Precision = 0.330 vs. 0.280
Recall = 0.828 vs. 1.000
Validation
AUC = 0.270 vs. 0.188
F1 = 0.357 vs. 0.317
Precision = 0.227 vs. 0.190
Recall = 0.843 vs. 0.963
Minibatch Loss= 5.716921
Train Batch Evaluation
AUC = 0.454 vs. 0.327
F1 = 0.513 vs. 0.477
Precision = 0.456 vs. 0.313
Recall = 0.585 vs. 0.996
Validation
AUC = 0.284 vs. 0.190
F1 = 0.362 vs. 0.318
Precision = 0.227 vs. 0.190
Recall = 0.893 vs. 0.973
Minibatch Loss= 5.342361
Train Batch Evaluation
AUC = 0.502 vs. 0.334
F1 = 0.512 vs. 0.480
Precision = 0.394 vs. 0.316
Recall = 0.730 vs. 0.992
Validation
AUC = 0.313 vs. 0.188
F1 = 0.368 vs. 0.317
Precision = 0.239 vs. 0.188
Recall = 0.799 vs. 1.000
Minibatch Loss= 4.75

In [18]:
#Save the model
saver = tf.train.Saver()
saver.save(sess, checkpoint_path, global_step=trainer.global_step)

'../model/CNN_cpd_64/CNN_cpd_64-20000'

In [31]:
#load the model
saver = tf.train.Saver()
#saver.restore(sess, "../model" + "/CNN7002_Gleam" + "/CNN7002_Gleam")
saver.restore(sess, '../model/CNN9006_opp/CNN9006_opp-20000')


INFO:tensorflow:Restoring parameters from ../model/CNN9006_opp/CNN9006_opp-20000


In [21]:
#This is the model evaluation on test dataset
result= nn_eval.evaluate_model (model, sess, test_data, test_gt)

AUC = 0.608 vs. 0.193
F1 = 0.605 vs. 0.317
Precision = 0.552 vs. 0.189
Recall = 0.668 vs. 1.000


In [22]:
probability_of_pos = nn_eval.deploy(model, sess, test_data, test_gt)

In [24]:
probability_of_pos.shape

(233, 16, 1)

In [25]:
test_data.shape

(233, 1024, 77)

In [26]:
#shape of probability_change is (batch size, T, num_classes)
#since num_classes = 1 we squeeze that axis
probability_change = np.squeeze(probability_of_pos, axis = 2)

In [29]:
savemat('../poc/poc'+exp_name+'.mat', {'poc':probability_change})

In [ ]:
1